In [ ]:
# Classes list
classes = []

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
!mkdir "/mydrive/yolov3"

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp cfg/yolov3-voc.cfg cfg/yolov3_testing.cfg

'line_number s@current_line @ replaced_content@'

In [ ]:
cfg/yolov3_training.cfg
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg # max_batches = (number of classes) * 2000
!sed -i '22 s/steps=400000,45000/steps=3200,3600/' cfg/yolov3_training.cfg # Steps = (max_batches * 0.8), (max_batches * 0.9)
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg # filters=(classes + 5)x3
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# cfg/yolov3_testing.cfg
!sed -i '3 s/batch=1/batch=64/' cfg/yolov3_testing.cfg
!sed -i '4 s/subdivisions=1/subdivisions=16/' cfg/yolov3_testing.cfg
!sed -i '20 s/max_batches = 50200/max_batches = 4000/' cfg/yolov3_testing.cfg # max_batches = (number of classes) * 2000
!sed -i '22 s/steps=400000,45000/steps=3200,3600/' cfg/yolov3_testing.cfg # Steps = (max_batches * 0.8), (max_batches * 0.9)
!sed -i '611 s@classes=20@classes=1@' cfg/yolov3_testing.cfg
!sed -i '695 s@classes=20@classes=1@' cfg/yolov3_testing.cfg
!sed -i '779 s@classes=20@classes=1@' cfg/yolov3_testing.cfg
!sed -i '605 s@filters=75@filters=18@' cfg/yolov3_testing.cfg # filters=(classes + 5)x3
!sed -i '689 s@filters=75@filters=18@' cfg/yolov3_testing.cfg
!sed -i '773 s@filters=75@filters=18@' cfg/yolov3_testing.cfg

In [ ]:
!echo > data/obj.names
with open('data/obj.names', 'w') as file:
  for class_name in classes:
    print(class_name)
    file.write(class_name)
    file.write("\n")

# Save weights to Drive
#!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

# Save weights to darknet/backup
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = backup/' > cfg/obj.data

!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

Modify image's path accordingly

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Convert class names to object class

In [ ]:
import glob
import os
import re
txt_file_paths = glob.glob(r"data/obj/images/*.txt", recursive=True)

for i, file_path in enumerate(txt_file_paths):
    text_converted = []
    # get image size
    with open(file_path, "r") as img_txt:
        lines = img_txt.readlines()
        for line in lines:
            object_name = re.findall('[aA-zZ]+', line)
            numbers = re.findall("[0-9.]+", line)
            if numbers:
                # Define coordinates
                text = "{} {} {} {} {}".format(object_name[0], numbers[0], numbers[1], numbers[2], numbers[3])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.png", recursive=True)
print(images_list)

In [ ]:
# Split Training and Testing
train_img_list = []
test_img_list = []

train_80 = round(len(images_list) * 0.8)
test_20 = round(len(images_list) * 0.2)

for i in range(0, train_80):
  train_img_list.append(images_list[i])
for k in range(train_80, train_80+test_20):
  test_img_list.append(images_list[k])

print("Train Set {}: \n".format(len(train_img_list)), train_img_list)
print("Test Set {}: \n".format(len(test_img_list)), test_img_list)

In [ ]:
# Create train.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(train_img_list)) 
file.close() 

# Create text.txt file
file = open("data/test.txt", "w") 
file.write("\n".join(test_img_list)) 
file.close() 

**6) Start Training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show --name yolov3_results  --cache

The below cell is to resume training of a model

In [ ]:
# Weights saved in Google drive
#!./darknet detector train cfg/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

# Weights Saved in Darknet/backup
!./darknet detector train cfg/obj.data cfg/yolov3_training.cfg backup/yolov3_training_last.weights -dont_show --name yolov3_results  --cache

**7) Evaluate Training**

Tracking and visualizing metrics such as loss and accuracy

`--name flag when training`

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

Calculation mAP (Mean Average Precision)

In [ ]:
# Weights saved in Google drive
#!./darknet detector map cfg/obj.data cfg/yolov3-voc_training.cfg /mydrive/yolov3/yolov3_training_last.weights -iou_thresh 0.75 -points 0

# Weights saved in darknet/backup
!./darknet detector map cfg/obj.data cfg/yolov3-voc_training.cfg backup/yolov3_training_last.weights -iou_thresh 0.75 -points 0

**8) Start Testing**

Single image test

In [ ]:
# Weights saved in Google drive
#!./darknet detector test cfg/obj.data cfg/yolov3_training.cfg backup/yolo-training_last.weights

# Weights saved in darknet/backup
!./darknet detector test cfg/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights

Perform testing to the whole test set

In [ ]:
# Weights saved in Google drive
#!./darknet detector test cfg/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show -ext_output < data/test.txt > data/result.txt

# Weights saved in Google drive
!./darknet detector test cfg/obj.data cfg/yolov3_training.cfg backup/yolo-training_last.weights -dont_show -ext_output < data/test.txt > data/result.txt

**9) Visualize Result**

Video/Camera Stream

In [ ]:
# Weights saved in Google drive
#!./darknet detector demo cfg/obj.data cfg/yolov3-voc_training.cfg /mydrive/yolov3/yolov3_training_last.weights

# Weights saved in Google drive
!./darknet detector demo cfg/obj.data cfg/yolov3-voc_training.cfg backup/yolov3_training_last.weights

In [ ]:
import glob
from IPython.display import Image, display
result_file_path = 'data/result.txt'
with open(result_file_path, "r") as img_txt:
        lines = img_txt.readlines()
        print(lines)
        for line in lines:
          print(line)
          display(Image(filename=line))